In [268]:
#libraries
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine
from sklearn.preprocessing import normalize

In [275]:
#read the data 
recipes = pd.read_csv('../data/full_clean_data.csv')
recipes.shape

(19224, 14)

In [273]:
##create a new df for user input & concate it to the original dataframe
#user input: a = total, b = calories, c = carbohydrate, d = fat, e = protein.
def user_input(a, b, c, d, e):

    user_df = pd.DataFrame({'name': 'user', 'url': 'no', 'category': 'no', 'rating': 'no', 'rating_count': 'no', 'ingredients': 'no', 'total': a, 'servings': 'no', 'yield': 'no', 'calories': b, 'carbohydrates_g': c, 'fat_g': d,'protein_g': e,'ingredients_parsed': 'no'}, index=[19224])
    full_df = pd.concat([recipes, user_df], ignore_index = False, axis = 0) 
    #rename the index
    full_df.index.names = ['recipes_id']
    ##normalize the total, calories, carb, fat, protein features
    new_norm = full_df[['total', 'calories', 'carbohydrates_g', 'fat_g', 'protein_g']]
    full_norm = pd.DataFrame(normalize(new_norm, axis=0))
    full_norm.columns = new_norm.columns
    full_norm.index = new_norm.index
    #create a column for the index for the original data
    full_df['recipes_id'] = full_df.index
    #print(full_df.shape)
    #use the old function here
    allRecipes = pd.DataFrame(full_norm.index)
    allRecipes = allRecipes[allRecipes.recipes_id != recipes_id]
    allRecipes["distance"] = allRecipes["recipes_id"].apply(lambda x: cosine(full_norm.loc[recipes_id], full_norm.loc[19244]))
    Top3NRecommendation = allRecipes.sort_values(["distance"]).head(3).sort_values(by=['distance', 'recipes_id'])
     # sort by distance then recipe id, the smaller value of recipe id will be picked. 
    recipe_df = full_norm.set_index('recipes_id')
    recipes_id = [recipes_id]
    recipe_list = []
    for recipeid in Top3NRecommendation.recipes_id:
        recipes_id.append(recipeid)   # list of recipe id of selected recipe and recommended recipe
        recipe_list.append("{}  {}".format(recipeid, full_norm.at[recipeid, 'url']))
        recipe_list.append("{}  {}".format(recipeid, full_norm.at[recipeid, 'name']))
    return recipe_list
    

In [274]:
user_input(45, 300, 40, 9, 8)

(19225, 15)


UnboundLocalError: local variable 'recipes_id' referenced before assignment